In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from helper.modules import EpilepticSeizure, LeaveOneOut
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping

In [21]:
config = {
    "data_path": "Data/",
    "preprocess": True,
    "labels": 5,
    "add_noise": True,
    "oversampling": True,
    "undersampling": True
}

es = EpilepticSeizure(config)

X, y = es.process()

X.head()

Processing files in E: 100%|██████████| 100/100 [00:00<00:00, 1495.70it/s]


,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,Channel_10,Channel_11,...,Channel_169,Channel_170,Channel_171,Channel_172,Channel_173,Channel_174,Channel_175,Channel_176,Channel_177,Channel_178
0,22.0,35.0,45.0,69.0,74.0,79.0,78.0,66.0,43.0,33.0,...,-4.0,-18.0,-32.0,-47.0,-53.0,-48.0,-40.0,-17.0,-23.0,-32.0
1,-50.0,-53.0,-49.0,-35.0,-28.0,-15.0,-2.0,14.0,18.0,15.0,...,46.0,34.0,22.0,4.0,-18.0,-31.0,-27.0,-26.0,-21.0,-30.0
2,-60.0,-73.0,-70.0,-70.0,-51.0,-29.0,-11.0,-7.0,-11.0,-6.0,...,15.0,1.0,-21.0,-38.0,-44.0,-31.0,-17.0,4.0,35.0,59.0
3,63.0,52.0,45.0,46.0,45.0,38.0,18.0,3.0,-4.0,-8.0,...,-3.0,-22.0,-47.0,-68.0,-85.0,-92.0,-96.0,-83.0,-73.0,-66.0
4,-48.0,-35.0,-22.0,4.0,18.0,21.0,17.0,18.0,11.0,5.0,...,6.0,4.0,-5.0,-16.0,-29.0,-35.0,-21.0,3.0,35.0,66.0


In [ ]:
def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

def build_model(num_classes=1):
    tf.keras.backend.clear_session()
    model = Sequential()
    model.add(LSTM(256, input_shape=(1, 178), activation='relu', return_sequences=True))
    model.add(LSTM(128, activation='relu', return_sequences=True))
    model.add(LSTM(64, activation='relu', return_sequences=True))
    model.add(LSTM(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))

    lr_scheduler = LearningRateScheduler(scheduler)

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
test = LeaveOneOut(data, build_model, 14, 'porn_addiction')

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='loss', patience=5, verbose=1)

models = test.train(epoch = 5, batch_size = 1250, callbacks = [early_stopping])